This session introduces GPU computing in Julia.

## GPGPU

GPUs are ubiquitous in modern computers. Following are GPUs today's typical computer systems.

| NVIDIA GPUs         | Tesla K80                            | GTX 1080                                 | GT 650M                              |
|---------------------|----------------------------------------|-----------------------------------------|--------------------------------------|
|                     | ![Tesla M2090](nvidia_k80.jpg) | ![GTX 580](nvidia_gtx1080.jpg)    | ![GT 650M](nvidia_gt650m.jpg) |
| Computers           | servers, cluster                       | desktop                                 | laptop                               |
|                     | ![Server](gpu_server.jpg)       | ![Desktop](alienware-area51.png) | ![Laptop](macpro_inside.png)  |
| Main usage          | scientific computing                   | daily work, gaming                      | daily work                           |
| Memory              | 24 GB                                    | 8 GB                                   | 1GB                                  |
| Memory bandwidth    | 480 GB/sec                              | 320 GB/sec                               | 80GB/sec                             |
| Number of cores     | 4992                                    | 2560                                     | 384                                  |
| Processor clock     | 562 MHz                                 | 1.6 GHz                                  | 0.9GHz                               |
| Peak DP performance | 2.91 TFLOPS                              | 257 GFLOPS                                        |                                      |
| Peak SP performance | 8.73 TFLOPS                            | 8228 GFLOPS                              | 691Gflops                            |

GPU architecture vs CPU architecture.  
* GPUs contain 100s of processing cores on a single card; several cards can fit in a desktop PC  
* Each core carries out the same operations in parallel on different input data -- single program, multiple data (SPMD) paradigm  
* Extremely high arithmetic intensity *if* one can transfer the data onto and results off of the processors quickly

| ![i7 die](cpu_i7_die.png) | ![Fermi die](Fermi_Die.png) |
|----------------------------------|------------------------------------|
| ![Einstein](einstein.png) | ![Rain man](rainman.png)    |

## GPGPU in Julia

GPU support by Julia is under active development. Check [JuliaGPU](https://github.com/JuliaGPU) for currently available packages. 

There are multiple paradigms to program GPU in Julia, depending on the specific hardware.

- **CUDA** is an ecosystem exclusively for Nvidia GPUs. There are extensive CUDA libraries for scientific computing: CuBLAS, CuRAND, CuSparse, CuSolve, CuDNN, ...

  The [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl) package allows defining arrays on **Nvidia GPUs** and overloads many common operations.

- The [AMDGPU.jl](https://github.com/JuliaGPU/AMDGPU.jl) package allows defining arrays on **AMD GPUs** and overloads many common operations.

- The [Metal.jl](https://github.com/JuliaGPU/Metal.jl) package allows defining arrays on **Apple Silicon** and overloads many common operations.

- The [oneAPI.jl](https://github.com/JuliaGPU/oneAPI.jl) package allows defining arrays on **Intel GPUs** and overloads many common operations.

I'll illustrate using Metal.jl on my MacBook Pro running MacOS Ventura 13.2.1. It has Apple M2 chip with 38 GPU cores.

In [1]:
versioninfo()

Julia Version 1.8.5
Commit 17cfb8e65ea (2023-01-08 06:45 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin21.5.0)
  CPU: 12 × Apple M2 Max
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, apple-m1)
  Threads: 8 on 8 virtual cores
Environment:
  JULIA_NUM_THREADS = 8
  JULIA_EDITOR = code


Load packages:

In [2]:
using Pkg

Pkg.activate(pwd())
Pkg.instantiate()
Pkg.status()

  Activating project at `~/Documents/github.com/ucla-biostat-257/2023spring/slides/09-juliagpu`


Status `~/Documents/github.com/ucla-biostat-257/2023spring/slides/09-juliagpu/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [dde4c033] Metal v0.3.0
  [37e2e46d] LinearAlgebra


## Query GPU devices in the system

In [3]:
using Metal

Metal.versioninfo()

macOS 13.3.0, Darwin 21.5.0

Toolchain:
- Julia: 1.8.5
- LLVM: 13.0.1

1 device:
- Apple M2 Max (64.000 KiB allocated)


## Transfer data between main memory and GPU

In [4]:
# generate data on CPU
x = rand(Float32, 3, 3)
# transfer data form CPU to GPU
xd = MtlArray(x)

3×3 MtlMatrix{Float32}:
 0.631541   0.846448  0.379895
 0.813597   0.999113  0.253971
 0.0469926  0.179086  0.98002

In [5]:
# generate array on GPU directly
# yd = Metal.ones(3, 3)
yd = MtlArray(ones(Float32, 3, 3))

3×3 MtlMatrix{Float32}:
 1.0  1.0  1.0
 1.0  1.0  1.0
 1.0  1.0  1.0

In [6]:
# collect data from GPU to CPU
x = collect(xd)

3×3 Matrix{Float32}:
 0.631541   0.846448  0.379895
 0.813597   0.999113  0.253971
 0.0469926  0.179086  0.98002

## Linear algebra

In [7]:
using BenchmarkTools, LinearAlgebra, Random

Random.seed!(257)
n = 2^14
# on CPU
x = rand(Float32, n, n)
y = rand(Float32, n, n)
z = zeros(Float32, n, n)
# on GPU
xd = MtlArray(x)
yd = MtlArray(y)
zd = MtlArray(z)

# SP matrix multiplication on GPU
bm_gpu = @benchmark Metal.@sync mul!($zd, $xd, $yd)

BenchmarkTools.Trial: 6 samples with 1 evaluation.
 Range (min … max):  899.295 ms … 908.975 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     900.655 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   902.037 ms ±   3.630 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █ █   █   █           █                                     █  
  █▁█▁▁▁█▁▁▁█▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  899 ms           Histogram: frequency by time          909 ms <

 Memory estimate: 800 bytes, allocs estimate: 40.

For this problem size on this machine, we see GPU achieves a staggering **9 TFLOPS** throughput with single precision!

In [8]:
# SP throughput on GPU
(2n^3) / (minimum(bm_gpu.times) / 1e9)

9.78109311367398e12

In [9]:
# SP matrix multiplication on CPU
bm_cpu = @benchmark mul!($z, $x, $y)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 12.491 s (0.00% GC) to evaluate,
 with a memory estimate of 0 bytes, over 0 allocations.

In [10]:
# SP throughput on CPU
(2n^3) / (minimum(bm_cpu.times) / 1e9)

7.041877507800785e11

We see >10x speedup by GPUs in this matrix multiplication example.

In [11]:
# cholesky on Gram matrix
# This one doesn't seem to work on Apple M2 chip yet
# xtxd = xd'xd + I
# @benchmark Metal.@sync cholesky($(xtxd))

In [12]:
# xtx = collect(xtxd)
# @benchmark cholesky($(Symmetric(xtx)))

GPU speedup of Cholesky seems unavailable at the moment.

## Elementiwise operations on GPU

In [13]:
# elementwise function on GPU arrays
fill!(yd, 1)
bm_gpu = @benchmark Metal.@sync $zd .= log.($yd .+ sin.($xd))
bm_gpu

BenchmarkTools.Trial: 125 samples with 1 evaluation.
 Range (min … max):  39.877 ms … 52.636 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     39.904 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   40.009 ms ±  1.139 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

           ▁  ▇█▅▄▂   ▅  ▄▁▁                                   
  ▅▃▁▁▅▁▃▆██▅▆█████▆████████▆▁▅▅▃▃▃▃▁▁▃▃▁▁▃▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃ ▃
  39.9 ms         Histogram: frequency by time          40 ms <

 Memory estimate: 8.13 KiB, allocs estimate: 310.

In [14]:
# elementwise function on CPU arrays
x, y, z = collect(xd), collect(yd), collect(zd)
bm_cpu = @benchmark $z .= log.($y .+ sin.($x))
bm_cpu

BenchmarkTools.Trial: 3 samples with 1 evaluation.
 Range (min … max):  2.373 s …   2.422 s  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     2.407 s              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.401 s ± 24.741 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █                                       █               █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  2.37 s         Histogram: frequency by time        2.42 s <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [15]:
# Speed up
median(bm_cpu.times) / median(bm_gpu.times)

60.33024155872633

GPU brings great speedup (>50x) to the massive evaluation of elementary math functions.